#prequisites

In [1]:
# Set PROJECT_DIR
import os
from pathlib import Path
print("Current Working Directory:", Path.cwd())

!if [ -d "pdm" ]; then rm -rf pdm; fi
!git clone https://github.com/soroushdty/pdm.git

PROJECT_DIR = Path.cwd() / "pdm"
os.chdir(PROJECT_DIR)
print("PROJECT_DIR: ", Path.cwd())

Current Working Directory: /content
Cloning into 'pdm'...
remote: Enumerating objects: 727, done.
remote: Counting objects: 100% (72/72), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 727 (delta 38), reused 21 (delta 10), pack-reused 655 (from 2)
Receiving objects: 100% (727/727), 2.37 MiB | 19.88 MiB/s, done.
Resolving deltas: 100% (429/429), done.
PROJECT_DIR:  /content/pdm


In [2]:
# Install Requirements
from funcs.requirements_utils import install_missing
installed = install_missing(Path.cwd() / "requirements.txt", quiet=False)
print("Installed:", installed)

Installed: ['imbalanced-learn', 'iterative-stratification']


In [3]:
# Import
import numpy as np
import pandas as pd
from funcs.config_loader import load_config
from funcs.logger_setup import setup_logger
from funcs.preprocessing import run as preprocessing
from funcs.mean_pooling import mean_pooling
from funcs.sanitize_col_name import sanitize_col_name
from funcs. compute_embeddings import compute_embeddings
from funcs.train_ensemble_pipeline import train_ensemble_pipeline
from funcs.get_dataset_from_npz import get_dataset_from_npz

In [4]:
# Load config
CONFIG_PATH = Path.cwd() / "config.json"
cfg = load_config(CONFIG_PATH)
cfg

{'DIR_INPUT': 'input',
 'DIR_OUTPUT': 'output',
 'patient_col': 'Patient',
 'physician_col': 'Physician',
 'item_col': 'Item',
 'classes': ['Behavioral health',
  'Diagnoses',
  'Disabilities',
  'Infectious diseases',
  'Genetics',
  'Medications',
  'Sexual and reproductive health',
  'Social determinants of health',
  'Violence',
  'Other'],
 'other_class': ['Disabilities',
  'Infectious diseases',
  'Genetics',
  'Sexual and reproductive health',
  'Violence',
  'Social determinants of health'],
 'llms': ['sentence-transformers/all-MiniLM-L6-v2',
  'sentence-transformers/all-mpnet-base-v2',
  'sentence-transformers/multi-qa-MiniLM-L6-cos-v1',
  'pritamdeka/BioBERT-mnli-snli-scinli-scitail-mednli-stsb',
  'sentence-transformers/biomed-mpnet-base',
  'cambridgeltl/SapBERT-from-PubMedBERT-fulltext',
  'bert-base-uncased',
  'roberta-base',
  'distilbert-base-uncased',
  'nreimers/MiniLM-L6-H384-uncased',
  'emilyalsentzer/Bio_ClinicalBERT',
  'monologg/biobert_v1.1_pubmed',
  'bionlp/

In [5]:
# Start Logging
DIR_LOG = Path.cwd() / cfg["DIR_OUTPUT"] / "log.txt"
logger = setup_logger(DIR_LOG)

[08:56:45 02-02-26] INFO: Logging initialized at: /content/pdm/output/log.txt


# Preprocessing

In [6]:
p = preprocessing(cfg)
print('\n', p)

[08:56:59 02-02-26] INFO: [train] Class columns to average: ['Behavioral health', 'Diagnoses', 'Disabilities', 'Infectious diseases', 'Genetics', 'Medications', 'Sexual and reproductive health', 'Social determinants of health', 'Violence', 'Other']
[08:56:59 02-02-26] INFO: [train] Unique (Patient, Item) pairs found: 879
[08:57:01 02-02-26] INFO: [train] Rows before: 1758; rows after: 879; pairs merged: 879
[08:57:01 02-02-26] INFO: [test] Class columns to average: ['Behavioral health', 'Diagnoses', 'Disabilities', 'Infectious diseases', 'Genetics', 'Medications', 'Sexual and reproductive health', 'Social determinants of health', 'Violence', 'Other']
[08:57:01 02-02-26] INFO: [test] Unique (Patient, Item) pairs found: 120
[08:57:02 02-02-26] INFO: [test] Rows before: 240; rows after: 120; pairs merged: 120
[08:57:25 02-02-26] INFO: Saved item standardization map: /content/pdm/output/merge_map_train.json
[08:57:26 02-02-26] INFO: Saved item standardization map: /content/pdm/output/merge

In [7]:
train_csv = pd.read_csv(PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'train.csv')
test_csv = pd.read_csv(PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'test.csv')
train_map = PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'merge_map_train.json'
test_map = PROJECT_DIR / cfg ['DIR_OUTPUT'] / 'merge_map_test.json'
print('Shape of train.csv', train_csv.shape)
print('Shape of test.csv', test_csv.shape)
print('Path of train_map:', train_map)
print('Path of test_map:', test_map)

Shape of train.csv (868, 13)
Shape of test.csv (89, 13)
Path of train_map: /content/pdm/output/merge_map_train.json
Path of test_map: /content/pdm/output/merge_map_test.json


# y npz files generation

In [19]:
y_train

array([[0. , 0. , 0. , 0.5],
       [0. , 1. , 0. , 0. ],
       [0.5, 1. , 0. , 0. ],
       ...,
       [0. , 0.5, 0. , 0.5],
       [0. , 0. , 1. , 0. ],
       [0. , 0. , 1. , 0. ]])

In [32]:
col_included = [c for c in cfg['classes'] if c not in cfg['other_class']]

z_train = train_csv.loc[:, col_included]
yy_train = z_train.to_numpy()

z_test = test_csv.loc[:, col_included]
yy_test = z_test.to_numpy()


y_file_name = PROJECT_DIR / cfg['DIR_OUTPUT'] / 'y.npz'
np.savez_compressed(y_file_name, **{'train': yy_train, 'test': yy_test})
y_train = np.load(y_file_name)['train']
y_test = np.load(y_file_name)['test']
print("Shape of y_test:", y_train.shape)
print("Shape of y_train:",y_test.shape)

Shape of y_test: (868, 4)
Shape of y_train: (89, 4)


# embedding

In [25]:
llm_example = cfg['llms'][0]
BATCH_SIZE = cfg['BATCH_SIZE']
print(llm_example)
print(BATCH_SIZE)

sentence-transformers/all-MiniLM-L6-v2
128


In [33]:
items_train = train_csv['Item']
items_test = test_csv['Item']
items = pd.concat([items_train, items_test]).drop_duplicates().sort_values()
embedding = compute_embeddings(llm_example, items.to_list(), BATCH_SIZE)
print('Shape of embedding: ', embedding.shape)

Computing embeddings for sentence-transformers/all-MiniLM-L6-v2...
Attempting to load sentence-transformers/all-MiniLM-L6-v2 as SentenceTransformer...
[09:11:20 02-02-26] INFO: Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

Success with SentenceTransformer.
Shape of embedding:  (651, 384)


In [34]:
sanitized = sanitize_col_name(llm_example)
print('LLM: ', sanitized)
embedding_file_name = PROJECT_DIR / cfg['DIR_OUTPUT'] / f'embdedding-{sanitized}-item.npz'
item_embedding_dict = {item: embedding[i] for i, item in enumerate(items.to_list())}
np.savez_compressed(embedding_file_name, **item_embedding_dict)
print('Saved to: ', embedding_file_name)

LLM:  sentence_transformers__all_MiniLM_L6_v2
Saved to:  /content/pdm/output/embdedding-sentence_transformers__all_MiniLM_L6_v2-item.npz


# base model

In [11]:
# y_train_path = PROJECT_DIR / cfg['DIR_OUTPUT'] / 'y_train.npz'
# print('Path of y_train.npz: ', y_train_path)
# y_train = np.load(y_train_path)

# y_test_path = PROJECT_DIR / cfg['DIR_OUTPUT'] / 'y_test.npz'
# print('Path of y_test.npz: ', y_test_path)
# y_test = np.load(y_test_path)

# train_csv_path= PROJECT_DIR / cfg["DIR_OUTPUT"] / 'train.csv'
# print('Path of train.csv: ', train_csv_path)

# test_csv_path = PROJECT_DIR / cfg["DIR_OUTPUT"] / 'test.csv'
# print('Path of test.csv: ', test_csv_path)

# train_map = PROJECT_DIR / cfg["DIR_OUTPUT"] /'merge_map_train.json'
# print('Path of train_map.json', train_map)

# test_map = PROJECT_DIR / cfg["DIR_OUTPUT"] /'merge_map_test.json'
# print('Path of test_map.json', test_map)

Path of y_train.npz:  /content/pdm/output/y_train.npz
Path of y_test.npz:  /content/pdm/output/y_test.npz
Path of train.csv:  /content/pdm/output/train.csv
Path of test.csv:  /content/pdm/output/test.csv
Path of train_map.json /content/pdm/output/merge_map_train.json
Path of test_map.json /content/pdm/output/merge_map_test.json


In [12]:
# x_path = PROJECT_DIR / cfg["DIR_OUTPUT"] /'embdedding-sentence_transformers__all_MiniLM_L6_v2-item.npz'
# x = np.load(x_path)
# print('Path of x.npz: ', x_path)
# print("len(x.keys): ", len(x.keys()))

Path of x.npz:  /content/pdm/output/embdedding-sentence_transformers__all_MiniLM_L6_v2-item.npz
len(x.keys):  651


In [ ]:
# for k in x.keys():
#     arr = x[k]
#     print("NPZ key:", k, "type:", type(arr), "shape:", getattr(arr, "shape", None))
#     print("    overall var:", np.var(arr))

#     # Only compute per-feature variances if this looks like a 2D array
#     if hasattr(arr, "ndim") and arr.ndim == 2:
#         print("    first 5 feature vars:", np.var(arr, axis=0)[:5])
#     else:
#         print("    (skipping per-feature vars; ndim =", getattr(arr, "ndim", None), ")")

NPZ key: 27-Year-Old type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.100466e-05
    (skipping per-feature vars; ndim = 1 )
NPZ key: 40-Year-Old type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.0440709e-05
    (skipping per-feature vars; ndim = 1 )
NPZ key: 5 Weeks Obstetrical And Has Cramps type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.06307025e-05
    (skipping per-feature vars; ndim = 1 )
NPZ key: 53-Year-Old type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.060131e-05
    (skipping per-feature vars; ndim = 1 )
NPZ key: 61-Year-Old type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.0721157e-05
    (skipping per-feature vars; ndim = 1 )
NPZ key: Abdominal Pain type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.10338915e-05
    (skipping per-feature vars; ndim = 1 )
NPZ key: Abilify (Aripiprazole) 20 Mg type: <class 'numpy.ndarray'> shape: (384,)
    overall var: 1.0312047e-05
    (skipping per-feature v

In [13]:
# train_df = pd.read_csv(train_csv_path)
# print("Shape of train_df:", train_df.shape)

# test_df = pd.read_csv(test_csv_path)
# print("Shape of test_df:", test_df.shape)

Shape of train_df: (868, 13)
Shape of test_df: (89, 13)


In [16]:
# y_train = np.load(y_train_path)['0']
# y_test = np.load(y_test_path)['0']
# print("Shape of y_train:", y_train.shape)
# print("Shape of y_test:", y_test.shape)

# X_train, X_test = get_dataset_from_npz(x, y_train, y_test, train_map, test_map)
# print("Shape of X_train:", X_train.shape)
# print("Shape of X_test:", X_test.shape)

Shape of y_train: (868, 4)
Shape of y_test: (89, 4)
Shape of X_train: (868, 384)
Shape of X_test: (89, 384)


In [17]:
import logging, traceback
def main():
  try:
    print("Global X_train overall variance:", np.var(X_train))
    print("First 10 feature variances:", np.var(X_train, axis=0)[:10])
    train_ensemble_pipeline(X_train, y_train, X_test, y_test, cfg)
    print("Pipeline executed successfully.")
  except Exception as e:
    logging.info(f"CRITICAL ERROR: {e}")
    traceback.print_exc()


if __name__ == "__main__":
  main()

Global X_train overall variance: 0.0
First 10 feature variances: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[08:59:26 02-02-26] INFO: Starting Training on X_train: (868, 384)...
[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (462, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (462, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


...

[DEBUG] Preprocessor.fit: X shape: (463, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (463, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


[DEBUG] Preprocessor.fit: X shape: (694, 384)
[DEBUG] Preprocessor.fit: X_scaled shape: (694, 384)
[DEBUG] Preprocessor.fit: overall variance of X: 0.0
[DEBUG] Preprocessor.fit: overall variance of X_scaled: 0.0
[DEBUG] Preprocessor.fit: first 10 feature variances (scaled): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:648: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var
/usr/local/lib/python3.12/dist-packages/sklearn/decomposition/_pca.py:789: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


[09:00:39 02-02-26] INFO: Saved metrics to ./ensemble/train_cv_report/Train_CV_Metrics.csv
[09:00:41 02-02-26] INFO: Saved metrics to ./ensemble/Ensemble_Metrics.csv
[09:00:42 02-02-26] INFO: Pipeline Complete.
Pipeline executed successfully.
